In [314]:
import numpy as np
import pandas as pd
import heapq
import random

# Generation of initial population of random antibodies

In [315]:
# m: number of machines
# p: number of parts
# population_size : number of antibody of the initial population
# seed: seed for reproducible results
def generation_initial_population(p, m, population_size, seed):
    MaxCell = min(p,m) #calculation of max number of cells
    number_of_zeros = MaxCell - 1 #number of zeros in each antibody
    antibodies = np.empty(shape=(population_size, p+m+number_of_zeros), dtype=int)
    antibody = np.append(np.array([*range(1,p+m+1)]), np.zeros(number_of_zeros,dtype=int))
    for i in range(0,population_size):
        np.random.seed(seed) 
        np.random.shuffle(antibody) #random positions in the array
        antibodies[i] = antibody
    return antibodies

In [316]:
antibodies = generation_initial_population(p = 7, m = 5, population_size = 100, seed = 1995)
antibodies

array([[ 1,  8, 11, ...,  7,  3,  5],
       [ 1,  9,  6, ...,  0, 11,  0],
       [ 1, 10, 12, ...,  7,  6,  4],
       ...,
       [ 1,  4, 10, ...,  0,  9,  6],
       [ 1,  0,  0, ...,  7, 10, 12],
       [ 1,  3,  5, ...,  0,  0,  2]])

# Evaluate all existing antibodies and compute their affinities

Lectura de los datos del problema + Traducción del formato del dataset a matriz trabajo-estación

In [317]:
def part_machine_incidence_matrix(data):    
    f = open(data,'r')
    lines = [line.split('\n') for line in f]
    f.close()

    m,p = [int(num) for num in lines[8][0].split(' ')] #m: number of machines, p: number of parts

    machines=[[m,p]]
    for i in range(9,9+m):
        #machines.append([int(num) for num in lines[i][0].split(' ')])
        machines.append([int(lines[i][0].split(' ')[j]) for j in range(1,m)])

    columns, rows = ['M'+str(i) for i in range(1,m+1)], ['P'+str(i) for i in range(1,p+1)]
    m_p_matrix = pd.DataFrame(columns= columns, index= rows)

    ones_zeros = []
    for i in range(1,len(machines)):
        aux = []
        for j in range(1,p+1):
            if j in machines[i]: aux.append(1)
            else: aux.append(0)
        ones_zeros.append(aux)

    for i in range(0,len(columns)):
        m_p_matrix[columns[i]] = ones_zeros[i]

    return m_p_matrix, m, p, columns, rows

In [318]:
matrix, m, p, columns, rows = part_machine_incidence_matrix('data/instances/testset_a/5x7_Waghodekar_Sahu(1984)[Problem-2].txt')
matrix

,M1,M2,M3,M4,M5
P1,1,0,0,1,0
P2,0,1,0,1,1
P3,0,1,1,1,0
P4,0,1,1,1,1
P5,1,1,1,0,1
P6,1,0,1,0,1
P7,1,0,0,0,0


## Decodificación de anticuerpos
1er Paso: Separación de celdas.

In [319]:
def cell_identification(antibodies):
    total_cells = []
    for antibodie in antibodies:
        # print("antibodie", antibodie)
        flag = 1 #bandera que indica si el num anterior es un cero
        cells, cell = [],[]
        i = 0
        for num in antibodie:
            if (num == 0): 
                if flag == 0: 
                    i=i+1
                    cells.append(cell)
                    # print(cell)
                    cell = []
                flag = 1
            else: 
                cell.append(num)
                # print(num)
                flag = 0
                if num == antibodie[len(antibodie)-1]:
                    cells.append(cell)
        total_cells.append(cells)
        # print("cells", cells)
    # print("list of all cells for all antibodies",total_cells)
    # print("cells for antibody 1",total_cells[0])
    return total_cells

In [320]:
total_cells = cell_identification(antibodies = antibodies)
total_cells

[[[1, 8, 11], [12], [9, 10], [6, 2, 4, 7, 3, 5]],
 [[1, 9, 6, 3, 4, 2, 7, 10], [5, 12, 8], [11]],
 [[1, 10, 12, 11], [8], [5], [2, 9, 3, 7, 6, 4]],
 [[1], [2, 6, 3, 9, 7, 5], [4, 8, 10, 11], [12]],
 [[1, 5, 8, 12, 11, 10], [4], [9], [6, 7, 2, 3]],
 [[1], [9, 2, 6], [7, 4], [3, 10, 5, 12], [8, 11]],
 [[1, 4, 10, 8, 12, 5], [3, 11], [2, 7, 9, 6]],
 [[1], [9, 2], [7, 3, 11, 6, 5, 4, 8], [10, 12]],
 [[1, 3, 5, 10, 8, 4], [11, 6, 12], [9, 7], [2]],
 [[1, 11], [9], [7, 6, 12, 2, 4, 3, 10], [5, 8]],
 [[1, 6, 4, 5, 10, 3], [12, 2, 8], [11], [7], [9]],
 [[1, 12], [11, 7, 2, 8, 9, 3, 6, 5], [4, 10]],
 [[1, 2, 3, 4, 5, 6], [8, 9, 10, 11, 12], [7]],
 [[1, 8, 11], [12, 7, 9, 10], [6, 2, 4], [3, 5]],
 [[1, 9, 6, 3, 4, 2], [10], [5, 12, 8], [7, 11]],
 [[1, 10, 12, 11], [8, 7], [5], [2, 9, 3], [6, 4]],
 [[1], [2, 6, 3, 9], [5], [4, 8, 10, 11, 7, 12]],
 [[1, 5, 8, 12, 11, 10, 7], [4], [9], [6], [2, 3]],
 [[1], [9, 2, 6], [4], [3, 10, 5, 12, 7, 8, 11]],
 [[1, 4, 10, 8, 12, 5, 7], [3, 11], [2], [9, 6]],


reorganizar filas y columnas de la matriz maquinas-trabajos en función de lo descrito por el anticuerpo

In [321]:
def decode_cells(total_cells):
    total_machines, total_parts = [], []
    for antibodie in total_cells:
        # print("antibodie",antibodie)
        machines = []
        parts = []
        decoded_antibodie = antibodie
        for i in range(0,len(antibodie)):
            # print(antibodie[i])
            for j in range(0,len(antibodie[i])):
                if antibodie[i][j] <= p: 
                    parts.append(rows[antibodie[i][j]-1])
                    decoded_antibodie[i][j] = rows[antibodie[i][j]-1]
                else: 
                    machines.append(columns[antibodie[i][j]-p-1])
                    decoded_antibodie[i][j] = columns[antibodie[i][j]-p-1]
            antibodie = decoded_antibodie
        total_machines.append(machines)
        total_parts.append(parts)
        print("decoded antibodies",antibodie)

    return total_machines, total_parts
    # print(total_machines)
    # print(total_parts)

In [322]:
total_machines, total_parts = decode_cells(total_cells=total_cells)

decoded antibodies [['P1', 'M1', 'M4'], ['M5'], ['M2', 'M3'], ['P6', 'P2', 'P4', 'P7', 'P3', 'P5']]
decoded antibodies [['P1', 'M2', 'P6', 'P3', 'P4', 'P2', 'P7', 'M3'], ['P5', 'M5', 'M1'], ['M4']]
decoded antibodies [['P1', 'M3', 'M5', 'M4'], ['M1'], ['P5'], ['P2', 'M2', 'P3', 'P7', 'P6', 'P4']]
decoded antibodies [['P1'], ['P2', 'P6', 'P3', 'M2', 'P7', 'P5'], ['P4', 'M1', 'M3', 'M4'], ['M5']]
decoded antibodies [['P1', 'P5', 'M1', 'M5', 'M4', 'M3'], ['P4'], ['M2'], ['P6', 'P7', 'P2', 'P3']]
decoded antibodies [['P1'], ['M2', 'P2', 'P6'], ['P7', 'P4'], ['P3', 'M3', 'P5', 'M5'], ['M1', 'M4']]
decoded antibodies [['P1', 'P4', 'M3', 'M1', 'M5', 'P5'], ['P3', 'M4'], ['P2', 'P7', 'M2', 'P6']]
decoded antibodies [['P1'], ['M2', 'P2'], ['P7', 'P3', 'M4', 'P6', 'P5', 'P4', 'M1'], ['M3', 'M5']]
decoded antibodies [['P1', 'P3', 'P5', 'M3', 'M1', 'P4'], ['M4', 'P6', 'M5'], ['M2', 'P7'], ['P2']]
decoded antibodies [['P1', 'M4'], ['M2'], ['P7', 'P6', 'M5', 'P2', 'P4', 'P3', 'M3'], ['P5', 'M1']]
de

Representación de la matriz: usamos total_machines y total_parts, donde hemos colocado en orden las máquinas y los trabajos respectivamente.


In [323]:
def create_machine_part_matrix(matrix, antibodies, total_machines, total_parts):
    antibody_matrices = []
    for i in range(0,len(antibodies)):
        antibodie_matrix = matrix.loc[:,total_machines[i]]
        antibodie_matrix = antibodie_matrix.loc[total_parts[i]]
        antibody_matrices.append(antibodie_matrix)
    return antibody_matrices


antibody_matrices = create_machine_part_matrix(matrix=matrix,
                                                antibodies=antibodies, 
                                                total_machines=total_machines, 
                                                total_parts=total_parts)

In [324]:
def evaluate_antibodies(antibody_matrices, total_cells): #m and p should be added as parameters
    exceptions, voids, penalties = [],[],[]
    for i in range(0,len(total_cells)):
        # print("\n",total_cells[i])
        # print(antibody_matrices[i])
        void, exception, penalty = 0,0,0
        cell_machine_flag, cell_flag_part = 0,0
        for cell in total_cells[i]:
            # print(cell)
            machines, parts = [], []
            machine_flag, part_flag = 0,0
            for mp in cell:
                if mp[0] == 'M': 
                    machines.append(mp)
                    machine_flag = 1
                    # print(machines)
                if mp[0] == 'P': 
                    parts.append(mp)
                    part_flag = 1
                    # print(parts)
            if machine_flag == 0: cell_machine_flag = 1
            if part_flag == 0: cell_flag_part = 1
            # print(i, cell_machine_flag, cell_flag_part)
            for machine in machines:
                for part in antibody_matrices[i].index:
                    if part in parts and antibody_matrices[i][machine][part] == 0: 
                        void = void+1
                        # print("void",machine, part)
                    if part not in parts and antibody_matrices[i][machine][part] == 1:
                        exception = exception+1
                        # print("exception",machine, part)
            # print(void, exception)
            # print("\n")
        voids.append(void)
        exceptions.append(exception)
        penalty = 0.5 * (cell_machine_flag + cell_flag_part)
        # print(penalty)
        penalties.append(penalty)
    # print("voids",voids)
    # print("exceptions",exceptions)
    efficacies, affinities = [], []
    matrix_dimension = m*p
    for i in range(0,len(total_cells)):
        exceptions_ratio = exceptions[i]/matrix_dimension
        voids_ratio = voids[i]/matrix_dimension
        efficacy= (1-exceptions_ratio)/(1+voids_ratio)
        efficacies.append(efficacy)
        affinities.append(efficacy - efficacy * penalties[i])
    return efficacies, affinities, voids, exceptions


In [325]:
efficacies, affinities, voids, exceptions = evaluate_antibodies(antibody_matrices = antibody_matrices, total_cells=total_cells)
print(efficacies)
print(affinities)

[0.48571428571428577, 0.5609756097560975, 0.4871794871794872, 0.5263157894736842, 0.5263157894736842, 0.5135135135135136, 0.5750000000000001, 0.525, 0.525, 0.5897435897435898, 0.5405405405405405, 0.5581395348837209, 0.4285714285714286, 0.44736842105263164, 0.5263157894736842, 0.5135135135135136, 0.525, 0.5121951219512195, 0.5238095238095238, 0.5609756097560975, 0.6511627906976745, 0.6444444444444445, 0.6428571428571429, 0.6136363636363636, 0.6046511627906976, 0.7, 0.48571428571428577, 0.5609756097560975, 0.4871794871794872, 0.5263157894736842, 0.5263157894736842, 0.5135135135135136, 0.5750000000000001, 0.525, 0.525, 0.5897435897435898, 0.5405405405405405, 0.5581395348837209, 0.4285714285714286, 0.44736842105263164, 0.5263157894736842, 0.5135135135135136, 0.525, 0.5121951219512195, 0.5238095238095238, 0.5609756097560975, 0.6511627906976745, 0.6444444444444445, 0.6428571428571429, 0.6136363636363636, 0.6046511627906976, 0.7, 0.48571428571428577, 0.5609756097560975, 0.4871794871794872, 0.

# Select N% of antibodies with highest affinities & Clone selected antibodies

con el parámetro p de probabilities de np.random.choice podemos pasar un vector de probabilidades para el sampleo

In [344]:
lista = [1,5,4,2]
prob = [0.5,0.3,0.2,0]

In [353]:
from __future__ import division
import numpy as np
import random,pdb
import operator

def roulette_selection(weights):
        '''performs weighted selection or roulette wheel selection on a list
        and returns the index selected from the list'''

        # sort the weights in ascending order
        sorted_indexed_weights = sorted(enumerate(weights), key=operator.itemgetter(1));
        indices, sorted_weights = zip(*sorted_indexed_weights);
        # calculate the cumulative probability
        tot_sum=sum(sorted_weights)
        prob = [x/tot_sum for x in sorted_weights]
        cum_prob=np.cumsum(prob)
        # select a random a number in the range [0,1]
        random_num=random.random()

        for index_value, cum_prob_value in zip(indices,cum_prob):
            if random_num < cum_prob_value:
                return index_value

weights=[1,2,6,4,3,7,20]
print (roulette_selection(weights))
weights=[1,2,2,2,2,2,2]
print (roulette_selection(weights))

5
4


In [350]:
roulette_selection(lista)

AttributeError: 'int' object has no attribute 'fitness'

In [346]:
def weighted_random_choice(chromosomes):
    max = sum(chromosome.fitness for chromosome in chromosomes)
    pick = random.uniform(0, max)
    current = 0
    for chromosome in chromosomes:
        current += chromosome.fitness
        if current > pick:
            return chromosome

In [347]:
weighted_random_choice(lista)

AttributeError: 'int' object has no attribute 'fitness'

In [326]:
def antibodies_selection(antibodies, N, affinities):
    population_pool = len(antibodies)
    sel_probabilities = affinities/np.sum(affinities)
    size = round(population_pool*N)
    if np.count_nonzero(sel_probabilities != 0) < size:
        print(sel_probabilities)
        print(size)
        return print("¡¡ERROR!! en la selección inicial de anticuerpos. INSUFICIENTES ANTICUERPOS CON AFINIDAD > 1")
    else:
        positions_antibodies_selected = np.random.choice(population_pool, size=size, replace=False, p = sel_probabilities)
        antibodies_selected = antibodies[positions_antibodies_selected.tolist()]
        return antibodies_selected, positions_antibodies_selected

cloned_antibodies, positions_antibodies_selected = antibodies_selection(antibodies=antibodies, N=0.1, affinities= affinities)

print(positions_antibodies_selected)
cloned_antibodies

[42 74 20 75 84 22 70 76  6 97]


array([[ 1,  0,  2,  6,  3,  9,  0,  5,  0,  4,  8, 10, 11,  7, 12,  0],
       [ 1, 11,  0,  0,  9,  0,  0,  6, 12,  2,  4,  3, 10,  7,  5,  8],
       [ 1,  0,  0,  9,  2,  0,  0,  3, 11,  6,  5,  4,  8,  7, 10, 12],
       [ 1,  6,  4,  5, 10,  3,  7, 12,  2,  8,  0, 11,  0,  0,  0,  9],
       [ 1,  4, 10,  8, 12,  5,  0,  0,  3, 11,  0,  0,  2,  7,  9,  6],
       [ 1, 11,  0,  0,  9,  0,  0,  6, 12,  2,  4,  3, 10,  7,  5,  8],
       [ 1,  0,  9,  2,  6,  0,  0,  4,  0,  3, 10,  5, 12,  7,  8, 11],
       [ 1, 12,  0,  0,  0, 11,  0,  2,  8,  9,  3,  6,  5,  7,  4, 10],
       [ 1,  4, 10,  8, 12,  5,  0,  0,  3, 11,  0,  0,  2,  7,  9,  6],
       [ 1,  4, 10,  8, 12,  5,  7,  0,  3, 11,  0,  0,  2,  0,  9,  6]])

# Mutation operator

Aqui estoy comentiendo un fallo y es que realizo la mutación en todos los casos.
Se debe comparar la efficacy del anticuerpo antes y después de la mutación y quedarse con el mejor.

## Maturate cloned antibodies

In [327]:
def mutate_cloned_antibodies(cloned_antibodies):
    for antibodie in cloned_antibodies:
        # print(antibodie)
        positions = np.random.choice(len(antibodie),size=2,replace=False)
        # print(positions)
        antibodie[positions[0]], antibodie[positions[1]] = antibodie[positions[1]], antibodie[positions[0]]
    return cloned_antibodies

In [328]:
mutate_cloned_antibodies(cloned_antibodies = cloned_antibodies)

array([[ 1,  0,  2,  6, 12,  9,  0,  5,  0,  4,  8, 10, 11,  7,  3,  0],
       [ 1, 11,  0,  0,  5,  0,  0,  6, 12,  2,  4,  3, 10,  7,  9,  8],
       [ 1,  0,  0,  9,  2,  8,  0,  3, 11,  6,  5,  4,  0,  7, 10, 12],
       [ 1,  6, 12,  5, 10,  3,  7,  4,  2,  8,  0, 11,  0,  0,  0,  9],
       [ 1,  4, 10,  8, 12,  5,  7,  0,  3, 11,  0,  0,  2,  0,  9,  6],
       [ 1, 11,  0,  0,  9,  0,  0,  6, 12,  2,  4,  3, 10,  7,  5,  8],
       [ 1,  0,  9,  2,  6, 12,  0,  4,  0,  3, 10,  5,  0,  7,  8, 11],
       [ 1, 12,  0,  0,  0, 11,  8,  2,  0,  9,  3,  6,  5,  7,  4, 10],
       [ 1,  4, 10,  0, 12,  5,  0,  0,  3, 11,  8,  0,  2,  7,  9,  6],
       [ 1,  4, 10,  6, 12,  5,  7,  0,  3, 11,  0,  0,  2,  0,  9,  8]])

## Evaluate cloned antibodies

In [329]:
cloned_total_cells = cell_identification(cloned_antibodies)
cloned_total_cells

[[[1], [2, 6, 12, 9], [5], [4, 8, 10, 11, 7, 3]],
 [[1, 11], [5], [6, 12, 2, 4, 3, 10, 7, 9, 8]],
 [[1], [9, 2, 8], [3, 11, 6, 5, 4], [7, 10, 12]],
 [[1, 6, 12, 5, 10, 3, 7, 4, 2, 8], [11], [9]],
 [[1, 4, 10, 8, 12, 5, 7], [3, 11], [2], [9, 6]],
 [[1, 11], [9], [6, 12, 2, 4, 3, 10, 7, 5, 8]],
 [[1], [9, 2, 6, 12], [4], [3, 10, 5], [7, 8, 11]],
 [[1, 12], [11, 8, 2], [9, 3, 6, 5, 7, 4, 10]],
 [[1, 4, 10], [12, 5], [3, 11, 8], [2, 7, 9, 6]],
 [[1, 4, 10, 6, 12, 5, 7], [3, 11], [2], [9, 8]]]

In [330]:
cloned_total_machines, cloned_total_parts = decode_cells(cloned_total_cells)

decoded antibodies [['P1'], ['P2', 'P6', 'M5', 'M2'], ['P5'], ['P4', 'M1', 'M3', 'M4', 'P7', 'P3']]
decoded antibodies [['P1', 'M4'], ['P5'], ['P6', 'M5', 'P2', 'P4', 'P3', 'M3', 'P7', 'M2', 'M1']]
decoded antibodies [['P1'], ['M2', 'P2', 'M1'], ['P3', 'M4', 'P6', 'P5', 'P4'], ['P7', 'M3', 'M5']]
decoded antibodies [['P1', 'P6', 'M5', 'P5', 'M3', 'P3', 'P7', 'P4', 'P2', 'M1'], ['M4'], ['M2']]
decoded antibodies [['P1', 'P4', 'M3', 'M1', 'M5', 'P5', 'P7'], ['P3', 'M4'], ['P2'], ['M2', 'P6']]
decoded antibodies [['P1', 'M4'], ['M2'], ['P6', 'M5', 'P2', 'P4', 'P3', 'M3', 'P7', 'P5', 'M1']]
decoded antibodies [['P1'], ['M2', 'P2', 'P6', 'M5'], ['P4'], ['P3', 'M3', 'P5'], ['P7', 'M1', 'M4']]
decoded antibodies [['P1', 'M5'], ['M4', 'M1', 'P2'], ['M2', 'P3', 'P6', 'P5', 'P7', 'P4', 'M3']]
decoded antibodies [['P1', 'P4', 'M3'], ['M5', 'P5'], ['P3', 'M4', 'M1'], ['P2', 'P7', 'M2', 'P6']]
decoded antibodies [['P1', 'P4', 'M3', 'P6', 'M5', 'P5', 'P7'], ['P3', 'M4'], ['P2'], ['M2', 'M1']]


In [331]:
cloned_antibody_matrices = create_machine_part_matrix(matrix = matrix,
                                                        antibodies=cloned_antibodies, 
                                                        total_machines = cloned_total_machines, 
                                                        total_parts= cloned_total_parts)

In [332]:
cloned_efficacies, cloned_affinities, cloned_voids, cloned_exceptions = evaluate_antibodies(cloned_antibody_matrices,
                                                                                           cloned_total_cells)
print("Pool\n",antibodies)
print(efficacies)
print(affinities)

print("\nCloned\n",cloned_antibodies)
print(cloned_efficacies)
print(cloned_affinities)

Pool
 [[ 1  8 11 ...  7  3  5]
 [ 1  9  6 ...  0 11  0]
 [ 1 10 12 ...  7  6  4]
 ...
 [ 1  4 10 ...  0  9  6]
 [ 1  0  0 ...  7 10 12]
 [ 1  3  5 ...  0  0  2]]
[0.48571428571428577, 0.5609756097560975, 0.4871794871794872, 0.5263157894736842, 0.5263157894736842, 0.5135135135135136, 0.5750000000000001, 0.525, 0.525, 0.5897435897435898, 0.5405405405405405, 0.5581395348837209, 0.4285714285714286, 0.44736842105263164, 0.5263157894736842, 0.5135135135135136, 0.525, 0.5121951219512195, 0.5238095238095238, 0.5609756097560975, 0.6511627906976745, 0.6444444444444445, 0.6428571428571429, 0.6136363636363636, 0.6046511627906976, 0.7, 0.48571428571428577, 0.5609756097560975, 0.4871794871794872, 0.5263157894736842, 0.5263157894736842, 0.5135135135135136, 0.5750000000000001, 0.525, 0.525, 0.5897435897435898, 0.5405405405405405, 0.5581395348837209, 0.4285714285714286, 0.44736842105263164, 0.5263157894736842, 0.5135135135135136, 0.525, 0.5121951219512195, 0.5238095238095238, 0.5609756097560975, 0.6511

## Add R% of best cloned antibodies to the pool of antibodies

select the R% of best cloned antibodies and adds them to the pool of antibodies

In [333]:
positions_antibodies_selected

array([42, 74, 20, 75, 84, 22, 70, 76,  6, 97])

In [334]:
antibodies

array([[ 1,  8, 11, ...,  7,  3,  5],
       [ 1,  9,  6, ...,  0, 11,  0],
       [ 1, 10, 12, ...,  7,  6,  4],
       ...,
       [ 1,  4, 10, ...,  0,  9,  6],
       [ 1,  0,  0, ...,  7, 10, 12],
       [ 1,  3,  5, ...,  0,  0,  2]])

In [335]:
efficacies

[0.48571428571428577,
 0.5609756097560975,
 0.4871794871794872,
 0.5263157894736842,
 0.5263157894736842,
 0.5135135135135136,
 0.5750000000000001,
 0.525,
 0.525,
 0.5897435897435898,
 0.5405405405405405,
 0.5581395348837209,
 0.4285714285714286,
 0.44736842105263164,
 0.5263157894736842,
 0.5135135135135136,
 0.525,
 0.5121951219512195,
 0.5238095238095238,
 0.5609756097560975,
 0.6511627906976745,
 0.6444444444444445,
 0.6428571428571429,
 0.6136363636363636,
 0.6046511627906976,
 0.7,
 0.48571428571428577,
 0.5609756097560975,
 0.4871794871794872,
 0.5263157894736842,
 0.5263157894736842,
 0.5135135135135136,
 0.5750000000000001,
 0.525,
 0.525,
 0.5897435897435898,
 0.5405405405405405,
 0.5581395348837209,
 0.4285714285714286,
 0.44736842105263164,
 0.5263157894736842,
 0.5135135135135136,
 0.525,
 0.5121951219512195,
 0.5238095238095238,
 0.5609756097560975,
 0.6511627906976745,
 0.6444444444444445,
 0.6428571428571429,
 0.6136363636363636,
 0.6046511627906976,
 0.7,
 0.485714285

In [336]:
cloned_antibodies

array([[ 1,  0,  2,  6, 12,  9,  0,  5,  0,  4,  8, 10, 11,  7,  3,  0],
       [ 1, 11,  0,  0,  5,  0,  0,  6, 12,  2,  4,  3, 10,  7,  9,  8],
       [ 1,  0,  0,  9,  2,  8,  0,  3, 11,  6,  5,  4,  0,  7, 10, 12],
       [ 1,  6, 12,  5, 10,  3,  7,  4,  2,  8,  0, 11,  0,  0,  0,  9],
       [ 1,  4, 10,  8, 12,  5,  7,  0,  3, 11,  0,  0,  2,  0,  9,  6],
       [ 1, 11,  0,  0,  9,  0,  0,  6, 12,  2,  4,  3, 10,  7,  5,  8],
       [ 1,  0,  9,  2,  6, 12,  0,  4,  0,  3, 10,  5,  0,  7,  8, 11],
       [ 1, 12,  0,  0,  0, 11,  8,  2,  0,  9,  3,  6,  5,  7,  4, 10],
       [ 1,  4, 10,  0, 12,  5,  0,  0,  3, 11,  8,  0,  2,  7,  9,  6],
       [ 1,  4, 10,  6, 12,  5,  7,  0,  3, 11,  0,  0,  2,  0,  9,  8]])

In [337]:
cloned_efficacies

[0.5750000000000001,
 0.6136363636363636,
 0.44999999999999996,
 0.6136363636363636,
 0.5609756097560975,
 0.6428571428571429,
 0.5675675675675675,
 0.5750000000000001,
 0.4871794871794872,
 0.5641025641025641]

In [338]:
def select_best_cloned_antibodies(antibodies, cloned_antibodies, efficacies, 
                                    cloned_efficacies,
                                    affinities,
                                    cloned_affinities, 
                                    R, 
                                    positions_antibodies_selected):
    #if antibody efficacy improves after mutation, we keep the mutated antibody, otherwise we dismiss it.
    j = 0
    for i in range(0,len(cloned_antibodies)):
        if cloned_efficacies[i] < efficacies[(positions_antibodies_selected[i])]:
            print("clon {} presenta peor eficacia al ser mutado".format(cloned_antibodies[i]))
            print("eficacia clon mutado",cloned_efficacies[i])
            print("eficacia clon sin mutar",efficacies[(positions_antibodies_selected[i])])
            cloned_antibodies[i] = antibodies[(positions_antibodies_selected[i])]
            cloned_efficacies[i] = efficacies[(positions_antibodies_selected[i])]
            cloned_affinities[i] = affinities[(positions_antibodies_selected[i])]
            print("Conservamos el clon {} original".format(cloned_antibodies[i]))

    # second part of the function: select R% of the best (efficacy) cloned antibodies
    amount_selected_antibodies = round(len(cloned_efficacies)*R)
    print("{} antibodies were selected".format(amount_selected_antibodies))
    positions = [i #positions of best R% of selected antibodies
        for x, i
        in heapq.nlargest(
            amount_selected_antibodies,
            ((x, i) for i, x in enumerate(cloned_efficacies)))]
    
    for i in range(0,amount_selected_antibodies):
        antibodies[(positions_antibodies_selected[(positions[i])])] = cloned_antibodies[(positions[i])]
        efficacies[(positions_antibodies_selected[(positions[i])])] = cloned_efficacies[(positions[i])]
        affinities[(positions_antibodies_selected[(positions[i])])] = cloned_affinities[(positions[i])]
    return antibodies, efficacies, affinities

In [339]:
antibodies, efficacies, affinities = select_best_cloned_antibodies(antibodies = antibodies,
                                    cloned_antibodies = cloned_antibodies,
                                    efficacies = efficacies, 
                                    cloned_efficacies = cloned_efficacies,
                                    affinities = affinities,
                                    cloned_affinities = cloned_affinities,
                                    R = 0.5,
                                    positions_antibodies_selected = positions_antibodies_selected)
print(efficacies)
print(affinities)
antibodies


clon [ 1 11  0  0  5  0  0  6 12  2  4  3 10  7  9  8] presenta peor eficacia al ser mutado
eficacia clon mutado 0.6136363636363636
eficacia clon sin mutar 0.6428571428571429
Conservamos el clon [ 1 11  0  0  9  0  0  6 12  2  4  3 10  7  5  8] original
clon [ 1  0  0  9  2  8  0  3 11  6  5  4  0  7 10 12] presenta peor eficacia al ser mutado
eficacia clon mutado 0.44999999999999996
eficacia clon sin mutar 0.6511627906976745
Conservamos el clon [ 1  0  0  9  2  0  0  3 11  6  5  4  8  7 10 12] original
clon [ 1  4 10  8 12  5  7  0  3 11  0  0  2  0  9  6] presenta peor eficacia al ser mutado
eficacia clon mutado 0.5609756097560975
eficacia clon sin mutar 0.5750000000000001
Conservamos el clon [ 1  4 10  8 12  5  0  0  3 11  0  0  2  7  9  6] original
clon [ 1 12  0  0  0 11  8  2  0  9  3  6  5  7  4 10] presenta peor eficacia al ser mutado
eficacia clon mutado 0.5750000000000001
eficacia clon sin mutar 0.6046511627906976
Conservamos el clon [ 1 12  0  0  0 11  0  2  8  9  3  6  5  7

array([[ 1,  8, 11, ...,  7,  3,  5],
       [ 1,  9,  6, ...,  0, 11,  0],
       [ 1, 10, 12, ...,  7,  6,  4],
       ...,
       [ 1,  4, 10, ...,  0,  9,  6],
       [ 1,  0,  0, ...,  7, 10, 12],
       [ 1,  3,  5, ...,  0,  0,  2]])

# Remove worst members of the antibodies pool (RECEPTOR EDITING)

"After mutation processes, the
antibodies that have worse efficacy values are erased (worst
%B of the whole population)"

In [340]:
def receptor_editing(antibodies_pool,efficacies,affinities, B):
    amount_antibodies = round(len(efficacies)*B)
    print("Se han borrado {} anticuerpos".format(amount_antibodies))
    positions = [i
        for x, i
        in heapq.nsmallest(
            amount_antibodies,
            ((x, i) for i, x in enumerate(efficacies)))]
    # print(positions)
    antibodies_pool = np.delete(antibodies_pool, positions, axis=0)
    efficacies = np.delete(efficacies, positions, axis=0)
    affinities = np.delete(affinities, positions, axis=0)
    return antibodies_pool, efficacies, affinities, amount_antibodies

In [341]:
B = 0.5
antibodies, efficacies, affinities, amount_antibodies_erased = receptor_editing(antibodies_pool = antibodies, 
                                                                                efficacies = efficacies, 
                                                                                affinities=affinities, 
                                                                                B = B)
antibodies

Se han borrado 50 anticuerpos


array([[ 1,  9,  6,  3,  4,  2,  7, 10,  0,  5, 12,  8,  0,  0, 11,  0],
       [ 1,  4, 10,  8, 12,  5,  0,  0,  3, 11,  0,  0,  2,  7,  9,  6],
       [ 1, 11,  0,  0,  9,  0,  7,  6, 12,  2,  4,  3, 10,  0,  5,  8],
       [ 1,  6,  4,  5, 10,  3,  0, 12,  2,  8,  0, 11,  0,  7,  0,  9],
       [ 1, 12,  0,  0,  0, 11,  7,  2,  8,  9,  3,  6,  5,  0,  4, 10],
       [ 1,  4, 10,  8, 12,  5,  7,  0,  3, 11,  0,  0,  2,  0,  9,  6],
       [ 1,  0,  0,  9,  2,  0,  0,  3, 11,  6,  5,  4,  8,  7, 10, 12],
       [ 1,  3,  5, 10,  8,  4,  7, 11,  6, 12,  0,  0,  9,  0,  0,  2],
       [ 1, 11,  0,  0,  9,  0,  0,  6, 12,  2,  4,  3, 10,  7,  5,  8],
       [ 1,  6,  4,  5, 10,  3,  7, 12,  2,  8,  0, 11,  0,  0,  0,  9],
       [ 1, 12,  0,  0,  0, 11,  0,  2,  8,  9,  3,  6,  5,  7,  4, 10],
       [ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12,  0,  0,  0,  0],
       [ 1,  9,  6,  3,  4,  2,  7, 10,  0,  5, 12,  8,  0,  0, 11,  0],
       [ 1,  4, 10,  8, 12,  5,  0,  0,  3, 11,  0,

# New Random antibodies into the population

"Then, same percent of new
antibodies are randomly generated"

La pregunta es la misma cantidad de anticuerpos que hemos borrado en el apartado anterior o el mismo porcentaje (B) sobre la nueva población que se ha visto disminuida por el receptor editing.

In [342]:
number_new_random_antibodies = amount_antibodies_erased

new_random_antibodies = generation_initial_population(p = p, 
                                                    m = m, 
                                                    population_size = number_new_random_antibodies, 
                                                    seed = 1995)
print(new_random_antibodies)

antibodies = np.concatenate((antibodies, new_random_antibodies), axis = 0)
antibodies

[[ 1  8 11  0  0 12  0  9 10  0  6  2  4  7  3  5]
 [ 1  9  6  3  4  2  7 10  0  5 12  8  0  0 11  0]
 [ 1 10 12 11  0  8  0  0  5  0  2  9  3  7  6  4]
 [ 1  0  2  6  3  9  7  5  0  4  8 10 11  0 12  0]
 [ 1  5  8 12 11 10  0  0  4  0  9  0  6  7  2  3]
 [ 1  0  9  2  6  0  7  4  0  3 10  5 12  0  8 11]
 [ 1  4 10  8 12  5  0  0  3 11  0  0  2  7  9  6]
 [ 1  0  0  9  2  0  7  3 11  6  5  4  8  0 10 12]
 [ 1  3  5 10  8  4  0 11  6 12  0  0  9  7  0  2]
 [ 1 11  0  0  9  0  7  6 12  2  4  3 10  0  5  8]
 [ 1  6  4  5 10  3  0 12  2  8  0 11  0  7  0  9]
 [ 1 12  0  0  0 11  7  2  8  9  3  6  5  0  4 10]
 [ 1  2  3  4  5  6  0  8  9 10 11 12  0  7  0  0]
 [ 1  8 11  0  0 12  7  9 10  0  6  2  4  0  3  5]
 [ 1  9  6  3  4  2  0 10  0  5 12  8  0  7 11  0]
 [ 1 10 12 11  0  8  7  0  5  0  2  9  3  0  6  4]
 [ 1  0  2  6  3  9  0  5  0  4  8 10 11  7 12  0]
 [ 1  5  8 12 11 10  7  0  4  0  9  0  6  0  2  3]
 [ 1  0  9  2  6  0  0  4  0  3 10  5 12  7  8 11]
 [ 1  4 10  8 12  5  7  0  3 11

array([[ 1,  9,  6, ...,  0, 11,  0],
       [ 1,  4, 10, ...,  7,  9,  6],
       [ 1, 11,  0, ...,  0,  5,  8],
       ...,
       [ 1,  3,  5, ...,  0,  0,  2],
       [ 1, 11,  0, ...,  7,  5,  8],
       [ 1,  6,  4, ...,  0,  0,  9]])

# Stopping criteria